In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.style
matplotlib.style.use("seaborn")
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import textwrap
import seaborn as sn
sn.color_palette("hls", 17)
import scipy.stats as st
import math

import pingouin as pg
plt.rcParams["font.family"] = "Times New Roman"

In [2]:
import sys
import os
sys.path.append(os.path.join(".."))

from module.trend_analysis_functions import *
from module.topic_model_plus_class import Topic_Model_plus

C:\Users\srandrad\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py:287: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
C:\Users\srandrad\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py:287: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
C:\Users\srandrad\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py:287: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
C:\Users\srandrad\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py:287: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
C:\Users\srandrad\Anaconda3\lib\

In [5]:
list_of_attributes = ["Narrative"]
#file = os.path.join("data","SAFENET","SAFENET_1999_2021.csv")
file = os.path.join(os.path.dirname(os.getcwd()),"results", "safenet_topics_May-04-2022", 'preprocessed_data.csv')
extra_cols = ["Iteration", "Event Start Date", "Event Stop Date", "Incident Name",
              "Incident Number", "Event State", "Event Jurisdiction", "Event Local Unit",
              "Event Position Title", "Event Task", "Event Management Level", "Event Resources", 
              "Event Incident Type", "Event Incident Activity", "Event Incident Stage", "Contributing Factors",
              "Human Factors", "Other Factors", "Narrative", "Immediate Action Taken",  "SAFENET Create Date"
              ]
document_id_col = "ID"
safenet = Topic_Model_plus(list_of_attributes=list_of_attributes, document_id_col=document_id_col, 
                        csv_file="", database_name="safenet", extra_cols=extra_cols)
safenet.extract_preprocessed_data(file)
df = safenet.data_df
print(len(df))
df = df.dropna(how='all', subset=['Narrative']).reset_index(drop=True)
print(len(df))

2361
2361


In [6]:
df['Year'] = [df.at[i,'Event Start Date'].split('/')[-1].split(" ")[0] for i in range(len(df))]
df['Day'] = [df.at[i,'Event Start Date'].split('/')[1] for i in range(len(df))]
df['Month'] = [df.at[i,'Event Start Date'].split('/')[0] for i in range(len(df))]

In [9]:
hazard_file = os.path.join(os.path.dirname(os.getcwd()),"results", "safenet_topics_May-04-2022", 'hazard_interpretation.xlsx')
results_file = os.path.join(os.path.dirname(os.getcwd()),"results", "safenet_topics_May-04-2022",'lda_results.xlsx')

In [10]:
frequency, docs = identify_docs_per_hazard(hazard_file, df, results_file, 'Narrative', 'Year', "ID", 'Narrative', doc_topic_dist_field='document topic distribution', topic_thresh=0.2,  ids_to_drop=[])

IndexError: single positional indexer is out-of-bounds

In [ ]:
def create_table(docs, frequency, preprocessed_df, id_field, categories, subcategories, hazards, time_field):#, accuracy_file):
    table = pd.DataFrame({"Category": categories, "Subcategory": subcategories, "Hazards": hazards})
    #accuracy_df = pd.read_excel(accuracy_file, sheet_name=['Summary'])['Summary']
    time_period = preprocessed_df[time_field].unique()
    severities = {name:{str(time_p):[] for time_p in time_period} for name in hazards}
    rates = {name:{str(time_p):0 for time_p in time_period} for name in hazards}
    total_docs_per_year = preprocessed_df[time_field].value_counts()
    total_rates = {hazard:0 for hazard in hazards}
    total_hazard_freq = {hazard:0 for hazard in hazards}
    total_severities_hazard = {hazard:0 for hazard in hazards}
    total_docs = len(preprocessed_df)
    #precision = []
    for hazard in hazards:
        #hazard_precision = accuracy_df.loc[accuracy_df['Hazards']==hazard].reset_index(drop=True)['Accuracy'].tolist()[0]
        #precision.append(round(hazard_precision,3))
        for year in docs[hazard]:
            year = str(year)
            for doc in docs[hazard][year]:
                doc_df = preprocessed_df.loc[preprocessed_df[id_field] == doc].reset_index(drop=True)
                severities[hazard][year].append(safecom_severity(doc_df.iloc[0]['Persons Onboard'], doc_df.iloc[0]['Injuries'], doc_df.iloc[0]['Damages']))
            rates[hazard][year] = frequency[hazard][year]/total_docs_per_year[year]
            total_hazard_freq[hazard] += frequency[hazard][year]
        total_rates[hazard] = round(total_hazard_freq[hazard]/len(time_period), 3)
        total_severities_hazard[hazard] = round(np.average([sev for year in severities[hazard] for sev in severities[hazard][year]]),3)
    table["Frequency"] = [total_hazard_freq[hazard] for hazard in total_hazard_freq]
    table["Rate"] = [total_rates[hazard] for hazard in total_rates]
    #table["Severity"] = [total_severities_hazard[hazard] for hazard in total_severities_hazard]
    #table["Precision"] = precision
    return table#, severities, rates